## ⚙️ 1. Setup: Install Libraries and import dependencies

First, let's install the specific version of the Google Agent Development Kit (ADK) that this notebook is built with. Pinning the version ensures our code will always work as expected. We will also import all the necessary modules from the ADK and other libraries.

In [ ]:
!pip install google-adk==1.8.0 -q

## 🔑 2. Authentication: Configure Your API Key

Next, we need to securely provide our Google API key. This code will create a secure input prompt for you to paste your key. It then sets the key as an environment variable, which is the standard way the ADK authenticates your requests.

In [ ]:
import os
from getpass import getpass

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully!")

## 🧞 3. Define the Agent's Blueprint

This is where we define our first agent. We create a function that returns an Agent instance. We give it a name, model, a detailed instruction prompt telling it how to behave like an expert event planner, and a tools list containing the built-in Google Search tool.

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search

def create_event_planner_agent():
    """Create event planner agent"""
    return Agent(
        name="event_planner_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in generating full-day itineraries based on mood, interests, and budget.",
        instruction="""
        You are an expert event planner. Your goal is to create a comprehensive plan for any event requested by the user using your search tool.

        You should:
        1.  **Brainstorm Themes:** Suggest 2-3 creative themes for the event.
        2.  **Find Venues:** Use your search tool to find 3-5 potential venues that fit the event's requirements.
        3.  **Suggest a Schedule:** Propose a high-level schedule or agenda for the event.
        4.  **Summarize:** Present the information in a clear, organized format.

        """,
        tools=[google_search]
    )

event_planner_agent = create_event_planner_agent()
print(f"🧞 Agent '{event_planner_agent.name}' is created and ready!")

# 🧠 4. Initialize Session Service

Before running any agents, we need to initialize a SessionService to manage their conversational memory. We'll use the InMemorySessionService for this notebook, which keeps track of conversations as long as the script is running.

In [ ]:
from google.adk.sessions import InMemorySessionService, Session

# --- Initialize our Session Service ---
session_service = InMemorySessionService()
user_id = "adk_event_planner_001"

# 🚀 5. Build the Execution Engine

This is our helper function to run queries. It handles the core ADK logic: initializing the Runner, streaming events with run_async, and displaying the final response. We'll reuse this function throughout our series.

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import Session
from google.genai.types import Content, Part
from IPython.display import display, Markdown

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"


    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response

# ✨ 6. Run the Agent

Finally, we bring everything together. This main function creates a new session for our conversation, defines the user's query, and then calls our run_agent_query helper to execute the agent and get a response.

In [ ]:
async def run_event_planner():
    # Create a new, single-use session for this query
    event_planning_session = await session_service.create_session(
        app_name=event_planner_agent.name,
        user_id=user_id
    )

    # Note the new budget constraint in the query!
    query = "Plan a small tech meetup for 30 people in New York on AI/ML. Budget under 5000$"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(event_planner_agent, query, event_planning_session, user_id)

await run_event_planner()